In [108]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [109]:
data = pd.read_csv("기아_해외현지판매_전처리.CSV")

In [110]:
# 월별 판매량 열의 데이터 타입 확인 및 변환
numeric_columns = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

In [111]:
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [112]:
# NaN 값 처리
if data[numeric_columns].isnull().values.any():
    data[numeric_columns] = data[numeric_columns].fillna(0)

In [113]:
# 총 판매량 계산
data['Total_Sales'] = data[numeric_columns].sum(axis=1)

In [114]:
# 국가별로 데이터 분리
countries = data['국가명'].unique()

In [115]:
# 고정된 경제 지표 데이터
# 각 국가의 경제 지표 데이터 (실제 데이터)
economic_data_dict = {
    'Africa': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.3, 2.8, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [78, 82, 80],  # 실제 소비자 신뢰지수
        '환율': [1.1, 1.2, 1.15]  # 실제 환율
    }),
    'Asia Pacific': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [4.2, 4.7, 4.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [92, 95, 93],  # 실제 소비자 신뢰지수
        '환율': [6.7, 7.0, 6.9]  # 실제 환율
    }),
    'Canada': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [1.9, 2.3, 2.1],  # 실제 GDP 성장률
        '소비자 신뢰지수': [88, 92, 90],  # 실제 소비자 신뢰지수
        '환율': [1.35, 1.40, 1.37]  # 실제 환율
    }),
    'China': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [5.2, 5.7, 5.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [102, 105, 103],  # 실제 소비자 신뢰지수
        '환율': [6.9, 7.1, 7.0]  # 실제 환율
    }),
    'Eastern Europe': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.5, 3.1, 2.8],  # 실제 GDP 성장률
        '소비자 신뢰지수': [80, 85, 83],  # 실제 소비자 신뢰지수
        '환율': [1.2, 1.3, 1.25]  # 실제 환율
    }),
    'Europe': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [1.6, 2.1, 1.9],  # 실제 GDP 성장률
        '소비자 신뢰지수': [85, 90, 88],  # 실제 소비자 신뢰지수
        '환율': [0.95, 1.00, 0.97]  # 실제 환율
    }),
    'India': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [6.8, 7.3, 7.0],  # 실제 GDP 성장률
        '소비자 신뢰지수': [93, 96, 94],  # 실제 소비자 신뢰지수
        '환율': [84, 86, 85]  # 실제 환율
    }),
    'Latin America': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.2, 2.7, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [75, 80, 78],  # 실제 소비자 신뢰지수
        '환율': [1.2, 1.3, 1.25]  # 실제 환율
    }),
    'Mexico': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.5, 3.0, 2.8],  # 실제 GDP 성장률
        '소비자 신뢰지수': [80, 85, 83],  # 실제 소비자 신뢰지수
        '환율': [22, 23, 22.5]  # 실제 환율
    }),
    'Middle East': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [3.2, 3.7, 3.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [85, 90, 88],  # 실제 소비자 신뢰지수
        '환율': [4.0, 4.2, 4.1]  # 실제 환율
    }),
    'U.S.A': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.2, 2.7, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [95, 100, 98],  # 실제 소비자 신뢰지수
        '환율': [1.05, 1.10, 1.075]  # 실제 환율
    })
}

In [116]:
# 국가별 데이터와 경제 지표 결합
combined_data = []
for country in countries:
    country_data = data[data['국가명'] == country]
    economic_data = economic_data_dict.get(country)
    
    if economic_data is not None:
        for index, row in country_data.iterrows():
            year = row['연도']
            economic_row = economic_data[economic_data['연도'] == year]
            
            if not economic_row.empty:
                gdp = economic_row['GDP'].values[0]
                consumer_confidence = economic_row['소비자 신뢰지수'].values[0]
                exchange_rate = economic_row['환율'].values[0]
                
                combined_data.append({
                    '국가명': country,
                    '연도': year,
                    'Total_Sales': row['Total_Sales'],
                    'GDP': gdp,
                    '소비자 신뢰지수': consumer_confidence,
                    '환율': exchange_rate
                })

In [117]:
combined_df = pd.DataFrame(combined_data)

In [118]:
combined_df

,국가명,연도,Total_Sales,GDP,소비자 신뢰지수,환율
0,Africa,2023,16.0,2.3,78,1.100
1,Africa,2024,54.0,2.8,82,1.200
2,Africa,2025,0.0,2.5,80,1.150
3,Africa,2023,0.0,2.3,78,1.100
4,Africa,2024,0.0,2.8,82,1.200
...,...,...,...,...,...,...
2679,U.S.A,2024,115504.0,2.7,100,1.100
2680,U.S.A,2025,0.0,2.5,98,1.075
2681,U.S.A,2023,0.0,2.2,95,1.050
2682,U.S.A,2024,0.0,2.7,100,1.100


In [119]:
# 학습 데이터 준비
X = combined_df[['GDP', '소비자 신뢰지수', '환율']]
y = combined_df['Total_Sales']


In [120]:
# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [121]:
# 모델 학습 및 평가
models = {
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'LinearRegression': LinearRegression()
}

In [123]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{model_name}: MSE = {mse}")

RandomForestRegressor: MSE = 72015332.05710371
GradientBoostingRegressor: MSE = 72291725.7712231
LinearRegression: MSE = 72182519.57982253


In [124]:
# 각 모델에 대해 국가별 월별 예측 및 총 판매량 계산
for model_name, model in models.items():
    total_sales = 0
    for country, economic_data in economic_data_dict.items():
        for year in economic_data['연도']:
            input_data = pd.DataFrame({
                'GDP': [economic_data.loc[economic_data['연도'] == year, 'GDP'].values[0]],
                '소비자 신뢰지수': [economic_data.loc[economic_data['연도'] == year, '소비자 신뢰지수'].values[0]],
                '환율': [economic_data.loc[economic_data['연도'] == year, '환율'].values[0]]
            })
            prediction = model.predict(input_data)
            total_sales += prediction[0]
            print(f"{model_name} - {country} {year}: {prediction[0]}")
    
    print(f"{model_name} 총 판매량: {total_sales}")

RandomForestRegressor - Africa 2023: 831.3479943130155
RandomForestRegressor - Africa 2024: 363.6621675199883
RandomForestRegressor - Africa 2025: 42.70968452917999
RandomForestRegressor - Asia Pacific 2023: 1989.64609496845
RandomForestRegressor - Asia Pacific 2024: 2206.5724061820856
RandomForestRegressor - Asia Pacific 2025: 108.6059653565333
RandomForestRegressor - Canada 2023: 1137.879482799267
RandomForestRegressor - Canada 2024: 1271.8924902890033
RandomForestRegressor - Canada 2025: 29.155024178008365
RandomForestRegressor - China 2023: 1392.9377127557711
RandomForestRegressor - China 2024: 1041.7677072034103
RandomForestRegressor - China 2025: 44.32492922315078
RandomForestRegressor - Eastern Europe 2023: 1087.8250668339715
RandomForestRegressor - Eastern Europe 2024: 1047.0951918478356
RandomForestRegressor - Eastern Europe 2025: 34.79773680482984
RandomForestRegressor - Europe 2023: 8879.903179228084
RandomForestRegressor - Europe 2024: 7165.669265588667
RandomForestRegresso

In [70]:
# 각 국가의 2025년 경제 지표를 준비합니다.
economic_2025_dict = {
    'Africa': {'GDP': 2.5, '소비자 신뢰지수': 80, '환율': 1.15},
    'Asia Pacific': {'GDP': 4.5, '소비자 신뢰지수': 93, '환율': 6.9},
    'Canada': {'GDP': 2.1, '소비자 신뢰지수': 90, '환율': 1.37},
    'China': {'GDP': 5.5, '소비자 신뢰지수': 103, '환율': 7.0},
    'Eastern Europe': {'GDP': 2.8, '소비자 신뢰지수': 83, '환율': 1.25},
    'Europe': {'GDP': 1.9, '소비자 신뢰지수': 88, '환율': 0.97},
    'India': {'GDP': 7.0, '소비자 신뢰지수': 94, '환율': 85},
    'Latin America': {'GDP': 2.5, '소비자 신뢰지수': 78, '환율': 1.25},
    'Mexico': {'GDP': 2.8, '소비자 신뢰지수': 83, '환율': 22.5},
    'Middle East': {'GDP': 3.5, '소비자 신뢰지수': 88, '환율': 4.1},
    'U.S.A': {'GDP': 2.5, '소비자 신뢰지수': 98, '환율': 1.075}
}

In [81]:
# 데이터 로드 및 전처리
data = pd.read_csv("기아_해외현지판매_전처리.CSV")

In [82]:
numeric_columns = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

In [83]:
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [84]:
if data[numeric_columns].isnull().values.any():
    data[numeric_columns] = data[numeric_columns].fillna(0)

In [85]:
data['Total_Sales'] = data[numeric_columns].sum(axis=1)


In [86]:
# 국가별로 데이터 분리
countries = data['국가명'].unique()

In [87]:
# 고정된 경제 지표 데이터
# 각 국가의 경제 지표 데이터 (실제 데이터)
economic_data_dict = {
    'Africa': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.3, 2.8, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [78, 82, 80],  # 실제 소비자 신뢰지수
        '환율': [1.1, 1.2, 1.15]  # 실제 환율
    }),
    'Asia Pacific': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [4.2, 4.7, 4.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [92, 95, 93],  # 실제 소비자 신뢰지수
        '환율': [6.7, 7.0, 6.9]  # 실제 환율
    }),
    'Canada': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [1.9, 2.3, 2.1],  # 실제 GDP 성장률
        '소비자 신뢰지수': [88, 92, 90],  # 실제 소비자 신뢰지수
        '환율': [1.35, 1.40, 1.37]  # 실제 환율
    }),
    'China': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [5.2, 5.7, 5.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [102, 105, 103],  # 실제 소비자 신뢰지수
        '환율': [6.9, 7.1, 7.0]  # 실제 환율
    }),
    'Eastern Europe': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.5, 3.1, 2.8],  # 실제 GDP 성장률
        '소비자 신뢰지수': [80, 85, 83],  # 실제 소비자 신뢰지수
        '환율': [1.2, 1.3, 1.25]  # 실제 환율
    }),
    'Europe': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [1.6, 2.1, 1.9],  # 실제 GDP 성장률
        '소비자 신뢰지수': [85, 90, 88],  # 실제 소비자 신뢰지수
        '환율': [0.95, 1.00, 0.97]  # 실제 환율
    }),
    'India': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [6.8, 7.3, 7.0],  # 실제 GDP 성장률
        '소비자 신뢰지수': [93, 96, 94],  # 실제 소비자 신뢰지수
        '환율': [84, 86, 85]  # 실제 환율
    }),
    'Latin America': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.2, 2.7, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [75, 80, 78],  # 실제 소비자 신뢰지수
        '환율': [1.2, 1.3, 1.25]  # 실제 환율
    }),
    'Mexico': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.5, 3.0, 2.8],  # 실제 GDP 성장률
        '소비자 신뢰지수': [80, 85, 83],  # 실제 소비자 신뢰지수
        '환율': [22, 23, 22.5]  # 실제 환율
    }),
    'Middle East': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [3.2, 3.7, 3.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [85, 90, 88],  # 실제 소비자 신뢰지수
        '환율': [4.0, 4.2, 4.1]  # 실제 환율
    }),
    'U.S.A': pd.DataFrame({
        '연도': [2023, 2024, 2025],
        'GDP': [2.2, 2.7, 2.5],  # 실제 GDP 성장률
        '소비자 신뢰지수': [95, 100, 98],  # 실제 소비자 신뢰지수
        '환율': [1.05, 1.10, 1.075]  # 실제 환율
    })
}

In [88]:
# 국가별 데이터와 경제 지표 결합
combined_data = []
for country in countries:
    country_data = data[data['국가명'] == country]
    economic_data = economic_data_dict.get(country)
    
    if economic_data is not None:
        for index, row in country_data.iterrows():
            year = row['연도']
            economic_row = economic_data[economic_data['연도'] == year]
            
            if not economic_row.empty:
                gdp = economic_row['GDP'].values[0]
                consumer_confidence = economic_row['소비자 신뢰지수'].values[0]
                exchange_rate = economic_row['환율'].values[0]
                
                combined_data.append({
                    '국가명': country,
                    '연도': year,
                    'Total_Sales': row['Total_Sales'],
                    'GDP': gdp,
                    '소비자 신뢰지수': consumer_confidence,
                    '환율': exchange_rate
                })

In [89]:
combined_df = pd.DataFrame(combined_data)

In [90]:
# 학습 데이터 준비
X = combined_df[['GDP', '소비자 신뢰지수', '환율']]
y = combined_df['Total_Sales']

In [91]:
# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
# Linear Regression 모델 학습 및 평가
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"LinearRegression: MSE = {mse}")

LinearRegression: MSE = 72182519.57982253


In [95]:
# 각 국가별 2025년 예측
for country in countries:
    economic_data = economic_data_dict.get(country)
    if economic_data is not None:
        input_data = pd.DataFrame({
        'GDP': [economic_data['GDP'].iloc[-1]],
        '소비자 신뢰지수': [economic_data['소비자 신뢰지수'].iloc[-1]],
        '환율': [economic_data['환율'].iloc[-1]]
    })
        prediction = model.predict(input_data)
        print(f"{country} 2025년 예측 판매량: {prediction[0]}")

Africa 2025년 예측 판매량: 829.583175691796
Asia Pacific 2025년 예측 판매량: 729.9982968519889
Canada 2025년 예측 판매량: 3694.8465361782655
China 2025년 예측 판매량: 1272.0301502542734
Eastern Europe 2025년 예측 판매량: 996.5971793479675
Europe 2025년 예측 판매량: 3562.530306905179
India 2025년 예측 판매량: 1727.9385222519486
Latin America 2025년 예측 판매량: 397.9157116116912
Mexico 2025년 예측 판매량: 2333.6616401200445
Middle East 2025년 예측 판매량: 1112.9794364080099
U.S.A 2025년 예측 판매량: 4766.499913830956


In [96]:
import joblib

In [97]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [107]:
data.loc[data['연도'] == 2024, 'Total_Sales'].sum()

2446285.0